In [16]:
import os
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.metrics import BinaryAccuracy

In [ ]:
train_dir = 'fire_data/train'
valid_dir = 'fire_data/valid'
test_dir = 'fire_data/test'

model_path = 'wildfire_detection_model.keras'

In [18]:
def load_data(directory, img_size=(32, 32)):
    X, Y = [], []
    for direct in os.listdir(directory):
        direct_path = os.path.join(directory, direct)
        if not os.path.isdir(direct_path):
            continue
        for filename in os.listdir(direct_path):
            img_path = os.path.join(direct_path, filename)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.resize(img, img_size)
            img = img.astype(np.float32) / 255.0
            X.append(img)
            Y.append(direct)
    return np.array(X), np.array(Y)

x_train, y_train = load_data(train_dir)
x_val, y_val = load_data(valid_dir)
x_test, y_test = load_data(test_dir)

# Convert labels to binary
y_train = np.array([1 if label == 'wildfire' else 0 for label in y_train])
y_val = np.array([1 if label == 'wildfire' else 0 for label in y_val])
y_test = np.array([1 if label == 'wildfire' else 0 for label in y_test])

print("x_train shape:", x_train.shape)
print("x_val shape:", x_val.shape)
print("x_test shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
print("y_test shape:", y_test.shape)

Premature end of JPEG file
Premature end of JPEG file


x_train shape: (30250, 32, 32, 3)
x_val shape: (6300, 32, 32, 3)
x_test shape: (6300, 32, 32, 3)
y_train shape: (30250,)
y_val shape: (6300,)
y_test shape: (6300,)


In [19]:
# Define the model
model = Sequential([
    Input(shape=(32, 32, 3)),  # Specify the input shape here
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Binary classification
])

In [22]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,829,189 (6.98 MB)

 Trainable params: 609,729 (2.33 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,219,460 (4.65 MB)

In [ ]:
# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[BinaryAccuracy(threshold=0.45)])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=30,
    batch_size=32,
    callbacks=[early_stopping]
)

# Save the model
model.save(model_path)

Epoch 1/30
946/946 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - binary_accuracy: 0.8785 - loss: 0.2922 - val_binary_accuracy: 0.9352 - val_loss: 0.1803
Epoch 2/30
946/946 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - binary_accuracy: 0.9293 - loss: 0.1865 - val_binary_accuracy: 0.9422 - val_loss: 0.1494
Epoch 3/30
946/946 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - binary_accuracy: 0.9334 - loss: 0.1684 - val_binary_accuracy: 0.9505 - val_loss: 0.1372
Epoch 4/30
946/946 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - binary_accuracy: 0.9429 - loss: 0.1513 - val_binary_accuracy: 0.9405 - val_loss: 0.1503
Epoch 5/30
946/946 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - binary_accuracy: 0.9414 - loss: 0.1477 - val_binary_accuracy: 0.9406 - val_loss: 0.1378
Epoch 6/30
946/946 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - binary_accuracy: 0.9455 - loss: 0.1393 - val_binary_accuracy: 0.9460 - val_loss: 0.1365
Epoch 7/30
946/946 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - binary_accuracy: 0.9515 - loss: 0.1257 - val_binary_accuracy: 0.9538 - val_loss:

In [21]:
# Predict on test data
test_predictions = model.predict(x_test)
y_pred = (test_predictions > 0.45).astype("int32").ravel()

# Evaluate with sklearn metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='binary')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Accuracy: 0.9604761904761905
Precision: 0.955969517358171
Recall: 0.9732758620689655
F1 Score: 0.964545066211021
